## pytorch lstm

- https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.7. 11</div>
<div style="text-align: right"> Last update: 2023.7.11</div>

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
# print(plt.stype.available)

# Options for pandas
pd.options.display.max_columns = 30

In [2]:
import torch
import torch.nn as nn

In [3]:
batch_size = 16
timesteps = 5
input_size = 10
output_size = 20
num_layers = 1

lstm = nn.LSTM(
    input_size = input_size, # 10
    hidden_size = output_size, # 20
    num_layers = num_layers, # 1
    batch_first = True   
)

In [4]:
input = torch.randn(batch_size, timesteps, input_size)
h0 = torch.randn(num_layers, batch_size, output_size)
c0 = torch.randn(num_layers, batch_size, output_size)

In [5]:
output, (hn, cn) = lstm(input, (h0, c0))

In [6]:
assert (batch_size, timesteps, output_size) == output.shape
assert (num_layers, batch_size, output_size) == hn.shape
assert (num_layers, batch_size, output_size) == cn.shape

In [7]:
print(output.shape)
print(hn.shape)
print(cn.shape)

torch.Size([16, 5, 20])
torch.Size([1, 16, 20])
torch.Size([1, 16, 20])
